In [1]:
import pandas as pd
import numpy as np
import json

In [2]:
df = pd.read_pickle('odin2018-2021')

In [3]:
df = df.rename(columns={'verttijd': 'prev_time'}) #TODO
df['seconds'] = 60*(df.datetime.dt.minute) +3600*(df.datetime.dt.hour) +df.datetime.dt.second
seconds_in_day = 24*60*60
df['sin_time'] = np.sin(2*np.pi*df.seconds/seconds_in_day)
df['cos_time'] = np.cos(2*np.pi*df.seconds/seconds_in_day)
df.drop(['seconds'], axis = 1, inplace = True)

In [4]:
with open('Explanations/otpexpl2018.json', "r") as json_file:
    choice_dict = json.load(json_file)

In [5]:
df.hvm = df.hvm.astype(int).astype(str)
with open('../Features_Dictionary', "r") as json_file:
    fd = json.load(json_file)
use_cols = [i for sublist in [fd[x] for x in fd.keys()] for i in sublist]
use_cols

['walk_distance',
 'bike_distance',
 'car_distance',
 'pt_distance',
 'walk_duration',
 'bike_duration',
 'car_duration',
 'pt_duration',
 'odin_duration',
 'odin_distance',
 'CanFelyx',
 'oprijbewijsmo',
 'oprijbewijsau',
 'geslacht',
 'leeftijd',
 'herkomst',
 'opleiding',
 'doel',
 'kmotiefv',
 'hhgestinkg',
 'hhauto',
 'prev_time',
 'vertpc',
 'aankpc',
 'sin_time',
 'cos_time',
 'weekdag',
 'feestdag',
 'jaar',
 'maand',
 'hvm',
 'khvm',
 'aank Man',
 'aank Huurwoning',
 'aank Age0',
 'aank Age1',
 'aank hh0',
 'aank hh1',
 'aank build_age0',
 'aank build_age1',
 'aank Immigration0',
 'aank Immigration1',
 'aank size0',
 'aank size1',
 'vert Man',
 'vert Huurwoning',
 'vert Age0',
 'vert Age1',
 'vert hh0',
 'vert hh1',
 'vert build_age0',
 'vert build_age1',
 'vert Immigration0',
 'vert Immigration1',
 'vert size0',
 'vert size1',
 'windspeed',
 'temp',
 'feelslike',
 'precip',
 'precipcover']

In [6]:
filter_cols =[col for col in use_cols if col in list(df.columns)]
df = df[filter_cols]
translate_cols = [col for col in use_cols if col in list(choice_dict.keys())]
translate_cols

['oprijbewijsmo',
 'oprijbewijsau',
 'geslacht',
 'leeftijd',
 'herkomst',
 'opleiding',
 'doel',
 'kmotiefv',
 'hhgestinkg',
 'hhauto',
 'weekdag',
 'feestdag',
 'maand',
 'hvm',
 'khvm']

In [7]:
for col in translate_cols: #if 2 cats make it int
    if col not in ['leeftijd', 'oprijbewijsau', 'oprijbewijsmo']:
        print(col)
        df[col] = df[col].astype(str)
        df = df.replace({col: choice_dict[col]})

geslacht
herkomst
opleiding
doel
kmotiefv
hhgestinkg
hhauto
weekdag
feestdag
maand
hvm
khvm


In [8]:
df[['vertpc', 'aankpc']] = df[['vertpc', 'aankpc']].astype(str)

In [9]:
df = df[df['odin_duration']<= 3600*1.5]

In [10]:
# felyxunknownint = ['hhpers', 'hhlft1', 'hhlft2', 'hhlft3', 'hhlft4', 'leeftijd', 'hhauto']
# felyxunknowncat = [x for x in felyxunknown if x not in felyxunknownint]
# df[felyxunknownint] = df[felyxunknownint].replace('Onbekend', '0')
# df[felyxunknownint] = df[felyxunknownint].astype(int)

In [11]:
df.dtypes

walk_distance    float64
bike_distance    float64
car_distance     float64
pt_distance      float64
walk_duration    float64
bike_duration    float64
car_duration     float64
pt_duration      float64
odin_duration    float64
odin_distance    float64
oprijbewijsmo      int64
oprijbewijsau      int64
geslacht          object
leeftijd           int64
herkomst          object
opleiding         object
doel              object
kmotiefv          object
hhgestinkg        object
hhauto            object
vertpc            object
aankpc            object
sin_time         float64
cos_time         float64
weekdag           object
feestdag          object
jaar              object
maand             object
hvm               object
khvm              object
windspeed        float64
temp             float64
feelslike        float64
precip           float64
precipcover      float64
dtype: object

In [12]:
# df.isna().sum()

In [13]:
# df[df.pt_dist.isna()][['vertpc', 'aankpc', 'walk_dist', 'bike_dist', 'car_dist', 'pt_dist']]

In [14]:
df['walk_distance'] = df['walk_distance'].fillna(df['bike_distance']) #High Confidence
df['walk_duration'] = df['walk_duration'].fillna(df['bike_duration']) #High Confidence

In [15]:
# df['pt_dur'] = df['pt_dur'].fillna(df['bike_dur']) #This is QUESTIONABLE
# df['pt_dist'] = df['pt_dist'].fillna(df['bike_dist']) #This is QUESTIONABLE
# df['car_dist'] = df['car_dist'].fillna(df['bike_dist']) #Medium ocnfidence
# df['car_dur'] = df['car_dur'].fillna(df['bike_dur']) #Medium ocnfidence

df = df.dropna()

In [16]:
pcdata = pd.read_pickle('../PostcodeInfo/PCData')
pcdata.index.names = ['pc4']
pcdata = pcdata.reset_index()

In [17]:
df.head()

,walk_distance,bike_distance,car_distance,pt_distance,walk_duration,bike_duration,car_duration,pt_duration,odin_duration,odin_distance,...,feestdag,jaar,maand,hvm,khvm,windspeed,temp,feelslike,precip,precipcover
verplid,,,,,,,,,,,,,,,,,,,,,
5924411224504,8250.0,8250.0,8257.0,14386.0,6241.0,1680.0,610.0,2336.0,900.0,10000.0,...,Nee,2020,Januari,Personenauto,Personenauto - bestuurder,34.5,6.1,2.1,0.0,0.0
5924411224505,8250.0,8250.0,8255.0,12441.0,6239.0,1679.0,607.0,1584.0,1140.0,10000.0,...,Nee,2020,Januari,Personenauto,Personenauto - bestuurder,34.5,6.1,2.1,0.0,0.0
5924411224506,8250.0,8250.0,8257.0,14386.0,6241.0,1680.0,610.0,2336.0,1800.0,15000.0,...,Nee,2020,Januari,Personenauto,Personenauto - bestuurder,34.5,6.1,2.1,0.0,0.0
5924436281803,2564.0,2595.0,4025.0,3514.0,2032.0,739.0,770.0,1580.0,360.0,3000.0,...,Nee,2020,Januari,Personenauto,Personenauto - bestuurder,34.5,6.1,2.1,0.0,0.0
5924436281804,2564.0,2565.0,2855.0,3460.0,2032.0,695.0,595.0,1602.0,360.0,3000.0,...,Nee,2020,Januari,Personenauto,Personenauto - bestuurder,34.5,6.1,2.1,0.0,0.0


In [18]:
df['CanFelyx'] = df['oprijbewijsau'] & df['oprijbewijsmo']

In [19]:
set((df.aankpc.unique())).difference(set(pcdata.pc4.unique()))  #TODO

{'1044',
 '1045',
 '1322',
 '1327',
 '1329',
 '3165',
 '3197',
 '3895',
 '5656',
 '9979'}

In [20]:
df = df.reset_index()

In [21]:
df = df.merge(pcdata.add_prefix('aank '), left_on='aankpc', right_on='aank pc4', how = 'inner')
len(df)

282828

In [22]:
df = df.merge(pcdata.add_prefix('vert '), left_on='vertpc', right_on='vert pc4', how = 'inner')
len(df)

282509

In [23]:
df = df.drop(['aank pc4', 'vert pc4'], axis=1)

In [24]:
df = df.set_index('verplid')

In [25]:
df['CanFelyx']

verplid
5924411224504    1
5924411224506    1
6228521441503    0
5924411224505    1
6228521441502    0
                ..
5583449869101    0
5583449869106    0
5583449869109    0
5592740404801    0
6522711165603    0
Name: CanFelyx, Length: 282509, dtype: int64

In [26]:
df.columns

Index(['walk_distance', 'bike_distance', 'car_distance', 'pt_distance',
       'walk_duration', 'bike_duration', 'car_duration', 'pt_duration',
       'odin_duration', 'odin_distance', 'oprijbewijsmo', 'oprijbewijsau',
       'geslacht', 'leeftijd', 'herkomst', 'opleiding', 'doel', 'kmotiefv',
       'hhgestinkg', 'hhauto', 'vertpc', 'aankpc', 'sin_time', 'cos_time',
       'weekdag', 'feestdag', 'jaar', 'maand', 'hvm', 'khvm', 'windspeed',
       'temp', 'feelslike', 'precip', 'precipcover', 'CanFelyx', 'aank Man',
       'aank Huurwoning', 'aank Age0', 'aank Age1', 'aank hh0', 'aank hh1',
       'aank build_age0', 'aank build_age1', 'aank Immigration0',
       'aank Immigration1', 'aank size0', 'aank size1', 'vert Man',
       'vert Huurwoning', 'vert Age0', 'vert Age1', 'vert hh0', 'vert hh1',
       'vert build_age0', 'vert build_age1', 'vert Immigration0',
       'vert Immigration1', 'vert size0', 'vert size1'],
      dtype='object')

In [27]:
df.to_pickle('OdinWrangled/Odin2018-2021All')

In [28]:
df = pd.read_pickle('OdinWrangled/Odin2018-2021All')

In [29]:
#2000-2037: Haarlem
DH = list(range(2490, 2599+ 1))
Rdam = list(range(3011,3089+ 1))
Adam = [str(x) for x in list(range(1011, 1109 + 1)) + list(range(1381, 1384 +1))]
Eind = list(range(5611,5658+1)) + list(range(5660,5667+1))#Geldrop
Tilb = list(range(5011,5049+1))
Breda = list(range(4800-4839+1))
HaaBloZan = list(range(2000, 2061+1))
Ensc = list(range(7511, 7548+1))
Urban = DH + Adam + Rdam + Eind + Tilb + Breda + HaaBloZan+ Ensc
Urban = [str(PC) for PC in Urban]

In [30]:
df = df[(df['aankpc'].isin(Urban)) | (df['vertpc'].isin(Urban))]
df.head()
df.to_pickle('OdinWrangled/Odin2018-2021UrbanOr')
len(df)

74646

In [31]:
df = df[(df['aankpc'].isin(Urban)) & (df['vertpc'].isin(Urban))]
df.head()
df.to_pickle('OdinWrangled/Odin2018-2021UrbanAnd')
len(df)

43115

In [32]:
df = df[(df['aankpc'].isin(Adam)) | (df['vertpc'].isin(Adam))]
df.to_pickle('OdinWrangled/Odin2018-2021Ams')
df.head()
len(df)

10831

In [33]:
for col in df.columns:
    print(col)

walk_distance
bike_distance
car_distance
pt_distance
walk_duration
bike_duration
car_duration
pt_duration
odin_duration
odin_distance
oprijbewijsmo
oprijbewijsau
geslacht
leeftijd
herkomst
opleiding
doel
kmotiefv
hhgestinkg
hhauto
vertpc
aankpc
sin_time
cos_time
weekdag
feestdag
jaar
maand
hvm
khvm
windspeed
temp
feelslike
precip
precipcover
CanFelyx
aank Man
aank Huurwoning
aank Age0
aank Age1
aank hh0
aank hh1
aank build_age0
aank build_age1
aank Immigration0
aank Immigration1
aank size0
aank size1
vert Man
vert Huurwoning
vert Age0
vert Age1
vert hh0
vert hh1
vert build_age0
vert build_age1
vert Immigration0
vert Immigration1
vert size0
vert size1


In [34]:
df.columns

Index(['walk_distance', 'bike_distance', 'car_distance', 'pt_distance',
       'walk_duration', 'bike_duration', 'car_duration', 'pt_duration',
       'odin_duration', 'odin_distance', 'oprijbewijsmo', 'oprijbewijsau',
       'geslacht', 'leeftijd', 'herkomst', 'opleiding', 'doel', 'kmotiefv',
       'hhgestinkg', 'hhauto', 'vertpc', 'aankpc', 'sin_time', 'cos_time',
       'weekdag', 'feestdag', 'jaar', 'maand', 'hvm', 'khvm', 'windspeed',
       'temp', 'feelslike', 'precip', 'precipcover', 'CanFelyx', 'aank Man',
       'aank Huurwoning', 'aank Age0', 'aank Age1', 'aank hh0', 'aank hh1',
       'aank build_age0', 'aank build_age1', 'aank Immigration0',
       'aank Immigration1', 'aank size0', 'aank size1', 'vert Man',
       'vert Huurwoning', 'vert Age0', 'vert Age1', 'vert hh0', 'vert hh1',
       'vert build_age0', 'vert build_age1', 'vert Immigration0',
       'vert Immigration1', 'vert size0', 'vert size1'],
      dtype='object')

In [166]:
df.head()

,walk_distance,bike_distance,car_distance,pt_distance,walk_duration,bike_duration,car_duration,pt_duration,odin_duration,odin_distance,...,vert Age0,vert Age1,vert hh0,vert hh1,vert build_age0,vert build_age1,vert Immigration0,vert Immigration1,vert size0,vert size1
verplid,,,,,,,,,,,,,,,,,,,,,
5924436012101,115442.0,115442.0,115599.0,108471.0,24454.0,24454.0,5791.0,6083.0,5400.0,92000.0,...,0.558821,0.608758,0.780162,0.029267,0.324323,0.136410,0.304127,0.289688,0.679605,0.629883
5908929840003,127723.0,127723.0,120844.0,125344.0,27415.0,27415.0,5719.0,7279.0,6900.0,125000.0,...,0.634387,0.488797,0.468565,0.036772,0.556998,0.136836,0.381816,0.348876,0.148876,0.433592
6203777226002,139061.0,139061.0,132641.0,130126.0,30026.0,30026.0,5767.0,8433.0,5400.0,115000.0,...,0.471155,0.495771,0.525805,0.034008,0.507096,0.106442,0.304127,0.289688,0.334911,0.371523
5896520918202,55262.0,55262.0,73904.0,61606.0,11792.0,11792.0,3273.0,4618.0,4800.0,100000.0,...,0.428045,0.553672,0.515782,0.038699,0.017276,0.050834,0.424018,0.112617,0.855664,0.300975
5617520695004,60623.0,60623.0,86469.0,69946.0,13142.0,13142.0,4115.0,5862.0,6000.0,80400.0,...,0.428045,0.553672,0.515782,0.038699,0.017276,0.050834,0.424018,0.112617,0.855664,0.300975


In [42]:
df.iloc[5, :]

index                                              125261
bike_dur                                          27415.0
bike_dist                                        127723.0
car_dur                                            5719.0
car_dist                                         120844.0
pt_dur                                             7279.0
pt_dist                                          125344.0
walk_dur                                          27415.0
walk_dist                                        127723.0
choice_dur                                         6900.0
choice_dist                                      125000.0
oprijbewijsau                                           1
geslacht                                              Man
leeftijd                                               62
herkomst                          Nederlandse achtergrond
opleiding            Hoger beroepsonderwijs, universiteit
doel                         Zakelijk bezoek in werksfeer
kmotiefv      

In [130]:
df.prev_time

AttributeError: 'DataFrame' object has no attribute 'prev_time'